In [1]:
import pandas as pd
df = pd.read_csv('melting_tank.csv')

In [2]:
df.drop(['NUM', 'INSP','MELT_WEIGHT'], axis = 1, inplace = True)

In [3]:
df['TAG'] = df['TAG'].replace({"OK" : 0, "NG" : 1})

# One Class SVM

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [5]:
df_co = df.copy()
df_co = df_co.sort_values(by = 'STD_DT')
df_co.head()

,STD_DT,MELT_TEMP,MOTORSPEED,TAG
0,2020-03-04 0:00,489,116,0
1,2020-03-04 0:00,433,78,0
2,2020-03-04 0:00,464,154,0
3,2020-03-04 0:00,379,212,0
4,2020-03-04 0:00,798,1736,0


In [6]:
df_co = df_co.set_index('STD_DT')
df_co.head()

,MELT_TEMP,MOTORSPEED,TAG
STD_DT,,,
2020-03-04 0:00,489,116,0
2020-03-04 0:00,433,78,0
2020-03-04 0:00,464,154,0
2020-03-04 0:00,379,212,0
2020-03-04 0:00,798,1736,0


In [7]:
train = df_co[df_co['TAG'] == 0]
train = train.iloc[:-210000]
test = pd.concat([df_co[df_co['TAG'] == 1],train.iloc[-210000:]])
#test = df_auto[df_auto['PassOrFail'] == 1]

print('train_value count: ')
print(train['TAG'].value_counts())
print('=============================')
print('=============================')
print('=============================')
print('test_value_count: ')
print(test['TAG'].value_counts())

X_train = train.drop('TAG', axis=1)
X_test = test.drop('TAG', axis=1)
print('=============================')
print(X_train.shape, X_test.shape)

train_value count: 
0    448133
Name: TAG, dtype: int64
test_value_count: 
0    210000
1    177067
Name: TAG, dtype: int64
(448133, 2) (387067, 2)


In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
from sklearn.covariance import EllipticEnvelope

In [10]:
ss = MinMaxScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [11]:
ee = EllipticEnvelope(contamination=0.01)
yhat = ee.fit_predict()

In [20]:
oc_svm_clf.fit(X_train)

OneClassSVM(gamma=0.01, kernel='linear', nu=0.1)

In [13]:
oc_svm_preds = oc_svm_clf.predict(X_test)

In [21]:
y_preds = []
for i in range(len(oc_svm_preds)):
    if oc_svm_preds[i] == -1:
        y_preds.append(1)
    else:
        y_preds.append(0)

In [22]:
from sklearn.metrics import classification_report

In [23]:
y_true = np.array((test['TAG']))
print(confusion_matrix(y_true, y_preds))
print(classification_report(y_true, y_preds))

[[195462  14538]
 [145289  31778]]
              precision    recall  f1-score   support

           0       0.57      0.93      0.71    210000
           1       0.69      0.18      0.28    177067

    accuracy                           0.59    387067
   macro avg       0.63      0.56      0.50    387067
weighted avg       0.63      0.59      0.52    387067

